<a href="https://colab.research.google.com/github/SalehMehrabani/PyTorch-Deep-Learning/blob/main/NN8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fraud Detection Model

* Imbalanced Dataset
* PyTorch
* Using Class Weightening


In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader,  Subset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [25]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [26]:
df= pd.read_csv("creditcard.csv")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [27]:
df.isna().sum().any()

False

In [28]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [29]:
df.shape

(284807, 31)

In [30]:
df['Class'].value_counts() 

0    284315
1       492
Name: Class, dtype: int64

In [31]:
NotFraudClassWeight = (492*100)/284807
fraudClassWeight = (284315*100)/284807

tensorWeights = torch.Tensor([fraudClassWeight,NotFraudClassWeight]).to(device)

fraudClassWeight, NotFraudClassWeight ,tensorWeights ,tensorWeights.device

(99.827251436938,
 0.1727485630620034,
 tensor([99.8273,  0.1727], device='cuda:0'),
 device(type='cuda', index=0))

In [32]:
#Split Data into Two Dataset
df_train, df_test = train_test_split(df, test_size=0.2, shuffle=True)

In [33]:
x = df_train.drop(["Class"],1)
y = df_train["Class"]

<ipython-input-33-54587342f702>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x = df_train.drop(["Class"],1)


In [34]:
#Convert the data frame to tensor
x =torch.from_numpy(x.values).type(dtype=torch.float).to(device)
y =torch.from_numpy(y.values).type(dtype=torch.float).to(device)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=True)

In [36]:
#Setup The Model 
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(nn.Linear(30, 64),
                                nn.ReLU(),
                                nn.Linear(64, 32),
                                nn.ReLU(),
                                nn.Linear(32, 16),
                                nn.ReLU(),
                                nn.Linear(16,2))
  

  def forward(self, x):
    x = self.layers(x)
    return x

In [37]:
model_0 = NeuralNetwork()
model_0 = model_0.to(device)
model_0

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=30, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=16, bias=True)
    (5): ReLU()
    (6): Linear(in_features=16, out_features=2, bias=True)
  )
)

In [38]:
#Setup The Loss Function
loss_fn= nn.CrossEntropyLoss(weight=tensorWeights, reduction='mean')

#Setup the Optimizer
optimizer= torch.optim.Adam(params=model_0.parameters(),lr=5e-5)

In [39]:
#Calculate the accuracy
def accuracy_fn(y_true, y_pred):
  correct=torch.eq(y_true, y_pred).sum().item()
  acc= (correct/len(y_pred))*100
  return acc

In [51]:
torch.manual_seed(42)

#Set the number of epoch
epochs=100
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

train_loss_values = []
test_loss_values = []
epoch_count = []

#Building training and evaluation loop:
for epoch in range(epochs):
  #Training
  model_0.train()

  ##Forward Pass
  y_pred = model_0(X_train)
  
  ##Claculate Loos/Accuracy
  loss = loss_fn(y_pred , y_train.long())

  ##Optimizer zero grad
  optimizer.zero_grad()

  ##Loss backward - Backward Propagation
  loss.backward()

  ##Optimizer Step
  optimizer.step()

  #Testing
  model_0.eval()
  with torch.inference_mode():

    ##Forward pass
    test_pred = model_0(X_test)

    ##Claculate Test Loos/ Accuracy
    test_loss = loss_fn(test_pred, y_test.long()) 


  #Print th result
  if epoch % 10 == 0:
    epoch_count.append(epoch)
    train_loss_values.append(loss.to("cpu").detach().numpy())
    test_loss_values.append(test_loss.to("cpu").detach().numpy())
    print(f"Epoch: {epoch} | Loss: {loss:.5f}| Test loss: {test_loss:.5f}")


Epoch: 0 | Loss: 0.00098| Test loss: 0.00104
Epoch: 10 | Loss: 0.00097| Test loss: 0.00103
Epoch: 20 | Loss: 0.00096| Test loss: 0.00102
Epoch: 30 | Loss: 0.00095| Test loss: 0.00101
Epoch: 40 | Loss: 0.00094| Test loss: 0.00100
Epoch: 50 | Loss: 0.00093| Test loss: 0.00099
Epoch: 60 | Loss: 0.00092| Test loss: 0.00098
Epoch: 70 | Loss: 0.00091| Test loss: 0.00097
Epoch: 80 | Loss: 0.00090| Test loss: 0.00096
Epoch: 90 | Loss: 0.00089| Test loss: 0.00095


In [52]:
x_final_test = df_test.drop(["Class"],1)
y_final_test = df_test["Class"]

<ipython-input-52-18f408227f83>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x_final_test = df_test.drop(["Class"],1)


In [53]:
x_final_test =torch.from_numpy(x_final_test.values).type(dtype=torch.float)
y_final_test =torch.from_numpy(y_final_test.values).type(dtype=torch.float)

In [58]:
torch.manual_seed(42)

#Set the number of epoch
epochs=1

x_final_test, y_final_test= x_final_test.to(device), y_final_test.to(device)

#Building training and evaluation loop:
for epoch in range(epochs):
  #Testing
  model_0.eval()
  with torch.inference_mode():

    ##Forward pass
    test_final_pred=model_0(x_final_test)

    ##Claculate Test Loos/ Accuracy
    test_final_loss = loss_fn(test_final_pred, y_final_test.long()) 

  #Print th result
  if epoch % 10 == 0:
    print(f"Epoch: {epoch} | Test loss: {test_final_loss:.5f}")

Epoch: 0 | Test loss: 0.00081
